In [1]:

# Installasi library yang dibutuhkan
# %pip install scikit-learn

# Import library yang diperlukan
import pandas as pd
from sklearn.utils import resample

# Memuat dataset dari file CSV
file_path = r'C:\Users\Nino\Documents\hateDetection-ETH\TweethateETHMU.csv'
data_raw = pd.read_csv(file_path, encoding='latin1', on_bad_lines='skip')

# Membersihkan nama kolom agar tidak ada karakter khusus
# seperti tanda kutip (") atau titik koma (;)
data_raw.columns = data_raw.columns.str.replace(r'[\";]', '', regex=True)

# Memilih kolom yang relevan untuk analisis
data_cleaned = data_raw[['full_text', 'usernamelabel']].copy()
data_cleaned.rename(columns={'full_text': 'text', 'usernamelabel': 'label'}, inplace=True)

# Mengekstrak label dari data
# Jika terdapat tanda titik koma (;), hanya mengambil bagian terakhir
data_cleaned['label'] = data_cleaned['label'].fillna('').astype(str)
data_cleaned['label'] = data_cleaned['label'].apply(lambda x: x.split(';')[-1] if ';' in x else x)

# Menghapus data duplikat berdasarkan teks dan label yang tidak valid
data_cleaned.drop_duplicates(subset='text', inplace=True)
data_cleaned = data_cleaned[data_cleaned['label'].isin(['hate', 'no-hate'])]

# Memeriksa distribusi awal kelas pada dataset
distribusi_awal = data_cleaned['label'].value_counts()
print("Distribusi Awal:")
print(distribusi_awal)

# Memisahkan data berdasarkan kelas (label)
no_hate = data_cleaned[data_cleaned['label'] == 'no-hate']
hate = data_cleaned[data_cleaned['label'] == 'hate']

# Melakukan oversampling pada kelas yang kurang
# Untuk kelas "hate", ditambah hingga memiliki 300 sampel
# Untuk kelas "no-hate", ditambah hingga memiliki 700 sampel
hate_upsampled = resample(hate, replace=True, n_samples=700, random_state=42)
no_hate_upsampled = resample(no_hate, replace=True, n_samples=700, random_state=42)

# Menggabungkan kembali data hasil oversampling
data_balanced = pd.concat([no_hate_upsampled, hate_upsampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# Memeriksa distribusi data setelah proses oversampling
distribusi_seimbang = data_balanced['label'].value_counts()
print("Distribusi Setelah Oversampling:")
print(distribusi_seimbang)

# Menyimpan dataset hasil oversampling ke file CSV
data_balanced.to_csv('OversampledV2_Tweet_Dataset.csv', index=False, encoding='utf-8')
print("Dataset disimpan sebagai 'Oversampled_Tweet_Dataset.csv'")


Distribusi Awal:
label
no-hate    257
hate        98
Name: count, dtype: int64
Distribusi Setelah Oversampling:
label
no-hate    700
hate       700
Name: count, dtype: int64
Dataset disimpan sebagai 'Oversampled_Tweet_Dataset.csv'
